<h1 style="text-align:center;">Stochastic Lotka–Volterra</h1>
<h3 style="text-align:center;">Stochastic Processes and Simulations in Natural Sciences (30561)</h3>
<h5 style="text-align:center;">Stoica Alexandru(3260547)   Dontu Catalin(3244174)   Antonescu Victor-Gabriel(3245562)</h3>

### Table of Contents:
1. Introduction
2. Libraries
3. The Deterministic Approach - ODE
4. The Stochastic Approach - Gillespie Simulation
5. Conclusions

### 1. Introduction

The Lotka-Voltera Predator-Pray Model is represented by 2 first-order nonlinear differential equations. It is often used in biological systems in which 2 species interact. The population of the 2 species evolves through time according to the following system:

$$
\begin{aligned}
\frac{dx(t)}{dt} &= \alpha x(t) - \beta x(t)y(t)\\[6pt]
\frac{dy(t)}{dt} &= -\gamma y(t) + \delta x(t)y(t) 
\end{aligned}
$$

$$
\alpha, \beta, \gamma, \delta \in \mathbb{R}_{+} \setminus \{0\}
$$

where  
- $x(t)$: prey population density at time t 
- $y(t)$: predator population density at time t 
- $\alpha$: prey's growth rate per capita
- $\beta$: effect of presence of predators on the prey's death rate
- $\gamma$: predator's death rate per capita
- $\delta$: effect of presence of pray on the predator's growth rate
- $t$: time

The model makes a number of **general assumptions** about the environment and biology of the predator and prey population:

- The prey population has an unlimited food supply at all times.
- The food supply of the predator population depends entierly on the size of the prey population.
- The rate of change of population is proportional to its size.
- The environment does not change during the process in favor of any species.
- Predators have limitless appettite.
- Populations are described by a single variable.

### 2. Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import log
from scipy.integrate import odeint

from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Output
import ipywidgets as widgets

import random
random.seed(42)

### 3. The Deterministic Approach - ODE 

The classic method of solving the predator pray model.

In [2]:
# Store initial values for reset
INITIAL_VALUES = {
    'b': 0.5,    # prey birth rate
    'm': 0.5,    # predator death rate
    'r': 0.02,   # prey death rate per predator
    'a': 0.01,   # predator birth rate per prey
    'H0': 40,    # initial prey
    'P0': 9,     # initial predators
    't_max': 200 # time span
}

EXTINCTION_THRESHOLD = 0.1

In [ ]:
# Model

def lotka_volterra(state, t, b, m, r, a):

    """
    Lotka-Volterra equations

    P = Predators, H = Prey

    Parameters:
    b: Birth rate of prey
    m: Death rate of predators
    r: Death rate of prey per predator
    a: Birth rate of predators per prey
    """

    H, P = state
    dH_dt = b*H - r*H*P
    dP_dt = -m*P + a*H*P
    return [dH_dt, dP_dt]

# Simulation

def simulate_lv(b, m, r, a, H0, P0, t_max, n_points=1000):

    """
    Solve the LV system and return t, H, P plus equilibrium values.
    """

    t = np.linspace(0, t_max, n_points)
    initial_state = [H0, P0]
    
    solution = odeint(lotka_volterra, initial_state, t, args=(b, m, r, a))
    H = solution[:, 0]
    P = solution[:, 1]
    
    # Equilibria 
    H_eq = m / a if a != 0 else 0
    P_eq = b / r if r != 0 else 0
    
    return t, H, P, H_eq, P_eq

In [ ]:
# Plotting

def extinction_message(H, P, threshold=EXTINCTION_THRESHOLD):
    H_extinct = np.any(H < threshold)
    P_extinct = np.any(P < threshold)

    if H_extinct and P_extinct:
        return "Both species go extinct!"
    elif H_extinct:
        return "Prey go extinct → Predators starve!"
    elif P_extinct:
        return "Predators go extinct → Prey explode!"
    return ""


def plot_lv(t, H, P, H_eq, P_eq, params, initial_state):
    """
    Draw the time series and phase-space plots.
    params: dict with keys 'b','m','r','a'
    initial_state: (H0, P0)
    """
    b, m, r, a = params['b'], params['m'], params['r'], params['a']
    H0, P0 = initial_state

    msg = extinction_message(H, P)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Time series
    ax1.plot(t, H, label='Prey (H)', linewidth=2)
    ax1.plot(t, P, label='Predators (P)', linewidth=2)

    if a > 0 and m > 0:
        ax1.axhline(y=H_eq, linestyle='--', alpha=0.5,
                    label=f'Prey eq: {H_eq:.1f}')
    if r > 0 and b > 0:
        ax1.axhline(y=P_eq, linestyle='--', alpha=0.5,
                    label=f'Pred eq: {P_eq:.1f}')

    title = 'Population Dynamics Over Time'
    if msg:
        title += f'\n{msg}'
    ax1.set_title(title)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Population')
    ax1.set_ylim(bottom=0)
    ax1.grid(True, alpha=0.3)
    ax1.legend()

    # Phase-space
    ax2.plot(H, P, linewidth=2)
    ax2.plot(H0, P0, 'ko', markersize=10, label=f'Initial ({H0:.0f}, {P0:.0f})')

    if a > 0 and m > 0 and r > 0 and b > 0:
        ax2.plot(H_eq, P_eq, 'ro', markersize=8,
                 label=f'Equilibrium ({H_eq:.1f}, {P_eq:.1f})')
        ax2.axvline(x=H_eq, linestyle=':', alpha=0.5)
        ax2.axhline(y=P_eq, linestyle=':', alpha=0.5)

    ax2.set_title('Phase Space (Predator vs Prey)')
    ax2.set_xlabel('Prey (H)')
    ax2.set_ylabel('Predators (P)')
    ax2.grid(True, alpha=0.3)
    ax2.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Widget Construction for Visualization

# Sliders
b_slider = FloatSlider(min=0.0, max=2.0, step=0.1,
                       value=INITIAL_VALUES['b'],
                       description='b (prey birth)', continuous_update=True)

m_slider = FloatSlider(min=0.0, max=2.0, step=0.1,
                       value=INITIAL_VALUES['m'],
                       description='m (pred death)', continuous_update=True)

r_slider = FloatSlider(min=0.0, max=0.1, step=0.005,
                       value=INITIAL_VALUES['r'],
                       description='r (prey death)', continuous_update=True)

a_slider = FloatSlider(min=0.0, max=0.1, step=0.005,
                       value=INITIAL_VALUES['a'],
                       description='a (pred birth)', continuous_update=True)

H0_slider = FloatSlider(min=0, max=100, step=5,
                        value=INITIAL_VALUES['H0'],
                        description='H0 (init prey)', continuous_update=True)

P0_slider = FloatSlider(min=0, max=100, step=2,
                        value=INITIAL_VALUES['P0'],
                        description='P0 (init pred)', continuous_update=True)

t_max_slider = FloatSlider(min=50, max=500, step=50,
                           value=INITIAL_VALUES['t_max'],
                           description='Time span', continuous_update=True)

# Buttons
only_prey_btn = Button(description="Only Prey Case",
                       button_style='info',
                       tooltip='Set parameters for prey-only scenario')

only_predators_btn = Button(description="Only Predators Case",
                            button_style='info',
                            tooltip='Set parameters for predator-only scenario')

equilibrium_btn = Button(description="Equilibrium Case",
                         button_style='success',
                         tooltip='Set classic parameters with equilibrium start')

reset_all_btn = Button(description="Reset All Parameters",
                       button_style='warning',
                       tooltip='Reset ALL parameters to initial values')

# Output area
output = Output()

# Layout
left_column = VBox([b_slider, r_slider, H0_slider])
right_column = VBox([m_slider, a_slider, P0_slider])

scenario_button_row = HBox(
    [only_prey_btn, equilibrium_btn, only_predators_btn],
    layout=widgets.Layout(justify_content='center', margin='5px 0')
)

utility_button_row = HBox(
    [reset_all_btn],
    layout=widgets.Layout(justify_content='center', margin='5px 0')
)

top_row = HBox([left_column, right_column])

bottom_row = VBox(
    [scenario_button_row, utility_button_row, t_max_slider],
    layout=widgets.Layout(align_items='center')
)

ui = VBox([top_row, bottom_row])

In [ ]:
# Callbacks

def read_params_from_sliders():
    return dict(
        b=b_slider.value,
        m=m_slider.value,
        r=r_slider.value,
        a=a_slider.value,
        H0=H0_slider.value,
        P0=P0_slider.value,
        t_max=t_max_slider.value,
    )

def plot_lotka_volterra(b, m, r, a, H0, P0, t_max):
    """Wrapper for interactive_output."""
    output.clear_output(wait=True)
    with output:
        t, H, P, H_eq, P_eq = simulate_lv(b, m, r, a, H0, P0, t_max)
        params = {'b': b, 'm': m, 'r': r, 'a': a}
        plot_lv(t, H, P, H_eq, P_eq, params, (H0, P0))


# Button actions
def only_prey_case(_):
    m_slider.value = 1.0
    a_slider.value = 0.0
    r_slider.value = 0.0
    P0_slider.value = 0.0

def only_predators_case(_):
    b_slider.value = 0.0
    H0_slider.value = 0.0
    r_slider.value = 0.0

def equilibrium_case(_):
    b_slider.value = 0.5
    m_slider.value = 0.5
    r_slider.value = 0.02
    a_slider.value = 0.01

    H_eq = m_slider.value / a_slider.value
    P_eq = b_slider.value / r_slider.value

    H0_slider.value = H_eq
    P0_slider.value = P_eq

def reset_initial_numbers(_):
    for k, v in INITIAL_VALUES.items():
        if k == 'b':   b_slider.value = v
        elif k == 'm': m_slider.value = v
        elif k == 'r': r_slider.value = v
        elif k == 'a': a_slider.value = v
        elif k == 'H0': H0_slider.value = v
        elif k == 'P0': P0_slider.value = v
        elif k == 't_max': t_max_slider.value = v

# Link buttons
only_prey_btn.on_click(only_prey_case)
only_predators_btn.on_click(only_predators_case)
equilibrium_btn.on_click(equilibrium_case)
reset_all_btn.on_click(reset_initial_numbers)

In [ ]:
# Initial Display

display(ui)

out = widgets.interactive_output(
    plot_lotka_volterra,
    {
        'b': b_slider,
        'm': m_slider,
        'r': r_slider,
        'a': a_slider,
        'H0': H0_slider,
        'P0': P0_slider,
        't_max': t_max_slider
    }
)

display(output)

# Initial plot
plot_lotka_volterra(**INITIAL_VALUES)

Output()

### 4. The Stochastic Approach - Gillespie Simulation

***Why Gillespie Algorithm?***

The dynamics of the predator-prey model can be expressed as discrete random events with known rates. This fits the assumpions of the Gillespie framework, that events occur one at a time, each with its own porbability rate. Markovianity is also checked by the fact that future evolution of the system depends only on the current state.

In result, the intial model is seen as a **Continuos Time Markov Jump Process**, modeled as following: